<a href="https://colab.research.google.com/github/angelrecalde2024/Power-System-Planning-and-Transmission-Design-2026/blob/main/INGP1118_EngineeringEconomics_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Engineering Economics (Money Concepts) for Power System Investments
# PV, FV, Annuity, CRF, Sinking Fund Deposit
# Power System Planning & Transmission Design

from dataclasses import dataclass
import math

@dataclass
class Project:
    i: float          # discount rate (decimal), e.g. 0.10
    T: int            # lifetime in years
    P0: float         # initial CAPEX at year 0 ($)
    reinv_year: int   # year of reinvestment
    reinv_cost: float # reinvestment cost at reinv_year ($)
    salvage: float    # salvage/residual value at year T ($)

def pv(amount: float, i: float, t: int) -> float:
    """Present value of amount received/paid at year t."""
    return amount / ((1 + i) ** t)

def fv_uniform_series(payment: float, i: float, n: int) -> float:
    """Future value at year n of uniform end-of-year payments (ordinary annuity)."""
    if i == 0:
        return payment * n
    return payment * (((1 + i) ** n - 1) / i)

def sinking_fund_deposit(target_fv: float, i: float, n: int) -> float:
    """Uniform end-of-year deposit required to reach target_fv at year n."""
    if i == 0:
        return target_fv / n
    return target_fv * (i / ((1 + i) ** n - 1))

def crf(i: float, T: int) -> float:
    """Capital Recovery Factor."""
    if i == 0:
        return 1 / T
    return (i * (1 + i) ** T) / ((1 + i) ** T - 1)

def annuity_from_present(P: float, i: float, T: int) -> float:
    """Equivalent uniform annual payment for present amount P over T years."""
    return P * crf(i, T)

def money_concepts_report(prj: Project) -> dict:
    """Compute key money-concept quantities."""
    # PVs of future items
    pv_reinv = pv(prj.reinv_cost, prj.i, prj.reinv_year)
    pv_salv  = pv(prj.salvage, prj.i, prj.T)

    # Simple "net present" of capex when accounting for salvage only (money concept)
    # (Not full NPV evaluation because we are not yet modeling benefits or O&M.)
    net_present_for_annualization = prj.P0 - pv_salv

    # Annualized CAPEX (annuity) using CRF
    A_capex_only = annuity_from_present(prj.P0, prj.i, prj.T)
    A_net_salv   = annuity_from_present(net_present_for_annualization, prj.i, prj.T)

    # Reserve fund deposit to accumulate reinvestment cost by reinv_year
    deposit = sinking_fund_deposit(prj.reinv_cost, prj.i, prj.reinv_year)
    check_fv = fv_uniform_series(deposit, prj.i, prj.reinv_year)

    return {
        "CRF": crf(prj.i, prj.T),
        "PV_reinvestment": pv_reinv,
        "PV_salvage": pv_salv,
        "Annualized_CAPEX_only": A_capex_only,
        "Annualized_CAPEX_net_salvage": A_net_salv,
        "Reserve_deposit_per_year_to_fund_reinvestment": deposit,
        "Check_FV_of_deposits_at_reinvestment_year": check_fv,
    }

def sensitivity(prj: Project, rates=(0.08, 0.10, 0.12), lifetimes=(20, 25, 30)) -> list:
    """Sensitivity of annualized CAPEX (net of salvage) to i and T."""
    rows = []
    for i in rates:
        for T in lifetimes:
            pv_salv = pv(prj.salvage, i, T)
            A_net = annuity_from_present(prj.P0 - pv_salv, i, T)
            rows.append((i, T, A_net))
    return rows

# -------------------------
# Default scenario (edit freely)
# -------------------------
P0 = 20_000_000.0
prj = Project(
    i=0.10,
    T=25,
    P0=P0,
    reinv_year=10,
    reinv_cost=0.30 * P0,  # 30% reinvestment
    salvage=0.10 * P0      # 10% salvage
)

rep = money_concepts_report(prj)

print("=== Money Concepts Report (Default Scenario) ===")
print(f"Discount rate i: {prj.i:.2%}")
print(f"Lifetime T: {prj.T} years")
print(f"Initial CAPEX P0: ${prj.P0:,.0f}")
print(f"Reinvestment C_{prj.reinv_year}: ${prj.reinv_cost:,.0f}")
print(f"Salvage S_{prj.T}: ${prj.salvage:,.0f}")
print()
print(f"CRF(i,T): {rep['CRF']:.6f}")
print(f"PV of reinvestment: ${rep['PV_reinvestment']:,.0f}")
print(f"PV of salvage: ${rep['PV_salvage']:,.0f}")
print()
print(f"Annualized CAPEX (CAPEX only): ${rep['Annualized_CAPEX_only']:,.0f} per year")
print(f"Annualized CAPEX (net of salvage PV): ${rep['Annualized_CAPEX_net_salvage']:,.0f} per year")
print()
print(f"Reserve deposit to fund reinvestment (years 1..{prj.reinv_year}): "
      f"${rep['Reserve_deposit_per_year_to_fund_reinvestment']:,.0f} per year")
print(f"Check FV at year {prj.reinv_year}: ${rep['Check_FV_of_deposits_at_reinvestment_year']:,.0f}")

print("\n=== Sensitivity: Annualized CAPEX (net of salvage PV) ===")
rows = sensitivity(prj)
for i, T, A_net in rows:
    print(f"i={i:.0%}, T={T:2d} -> Annualized CAPEX net salvage: ${A_net:,.0f}/yr")

=== Money Concepts Report (Default Scenario) ===
Discount rate i: 10.00%
Lifetime T: 25 years
Initial CAPEX P0: $20,000,000
Reinvestment C_10: $6,000,000
Salvage S_25: $2,000,000

CRF(i,T): 0.110168
PV of reinvestment: $2,313,260
PV of salvage: $184,592

Annualized CAPEX (CAPEX only): $2,203,361 per year
Annualized CAPEX (net of salvage PV): $2,183,025 per year

Reserve deposit to fund reinvestment (years 1..10): $376,472 per year
Check FV at year 10: $6,000,000

=== Sensitivity: Annualized CAPEX (net of salvage PV) ===
i=8%, T=20 -> Annualized CAPEX net salvage: $1,993,340/yr
i=8%, T=25 -> Annualized CAPEX net salvage: $1,846,218/yr
i=8%, T=30 -> Annualized CAPEX net salvage: $1,758,894/yr
i=10%, T=20 -> Annualized CAPEX net salvage: $2,314,273/yr
i=10%, T=25 -> Annualized CAPEX net salvage: $2,183,025/yr
i=10%, T=30 -> Annualized CAPEX net salvage: $2,109,426/yr
i=12%, T=20 -> Annualized CAPEX net salvage: $2,649,818/yr
i=12%, T=25 -> Annualized CAPEX net salvage: $2,534,999/yr
i=12%

In [2]:
# =========================
# ADD-ON CELL: Fixed O&M as an annual series (end-of-year payments)
# This cell assumes you already defined:
#   - prj (Project dataclass instance)
#   - pv(), crf(), annuity_from_present(), money_concepts_report() from the prior cell
# It will:
#   1) attach fixed O&M to the project as new variables (kept global for future cells)
#   2) compute PV of the fixed O&M series
#   3) compute Equivalent Uniform Annual Cost (EUAC-style) including:
#        - annualized CAPEX net of salvage PV
#        - + fixed O&M (already annual)
# =========================

# --- User-editable assumptions ---
fixed_om_per_year = 450_000.0     # $/year (end-of-year), edit freely
om_start_year = 1                # typically 1 (end of year 1)
om_end_year = prj.T              # through lifetime (end of year T)

# --- Helper: PV of a uniform annual series between years a..b (inclusive) ---
def pv_uniform_series(payment: float, i: float, a: int, b: int) -> float:
    """
    Present value at year 0 of uniform end-of-year payments from year a..b (inclusive).
    Example: a=1, b=T -> standard annuity PV.
    """
    if b < a:
        return 0.0
    if i == 0:
        return payment * (b - a + 1)
    return sum(payment / ((1 + i) ** t) for t in range(a, b + 1))

# --- Persist variables for future cells ---
fixed_om = fixed_om_per_year
om_years = (om_start_year, om_end_year)

# --- Compute PV of fixed O&M series ---
pv_fixed_om = pv_uniform_series(fixed_om, prj.i, om_start_year, om_end_year)

# --- Annualized CAPEX net of salvage PV (from money concepts) ---
pv_salv = pv(prj.salvage, prj.i, prj.T)
annualized_capex_net_salv = annuity_from_present(prj.P0 - pv_salv, prj.i, prj.T)

# --- Total Equivalent Uniform Annual Cost (EUAC-style) including fixed O&M ---
# Note: fixed O&M is already an annual amount; adding it to annualized CAPEX is consistent.
total_annual_cost = annualized_capex_net_salv + fixed_om

# --- Provide a compact report dictionary for reuse in later cells ---
econ_state = {
    "fixed_om_per_year": fixed_om,
    "om_start_year": om_start_year,
    "om_end_year": om_end_year,
    "PV_fixed_OM_series": pv_fixed_om,
    "PV_salvage": pv_salv,
    "Annualized_CAPEX_net_salvage": annualized_capex_net_salv,
    "Total_annual_cost_CAPEXplusFixedOM": total_annual_cost,
}

print("=== Added Fixed O&M as Annual Series ===")
print(f"Fixed O&M: ${fixed_om:,.0f}/yr (years {om_start_year}..{om_end_year}, end-of-year)")
print(f"PV of fixed O&M series: ${pv_fixed_om:,.0f}")
print(f"Annualized CAPEX (net salvage PV): ${annualized_capex_net_salv:,.0f}/yr")
print(f"TOTAL annual cost (CAPEX annuity + fixed O&M): ${total_annual_cost:,.0f}/yr")
print("\nVariables saved for future cells: fixed_om, om_years, pv_fixed_om, annualized_capex_net_salv, total_annual_cost, econ_state")

=== Added Fixed O&M as Annual Series ===
Fixed O&M: $450,000/yr (years 1..25, end-of-year)
PV of fixed O&M series: $4,084,668
Annualized CAPEX (net salvage PV): $2,183,025/yr
TOTAL annual cost (CAPEX annuity + fixed O&M): $2,633,025/yr

Variables saved for future cells: fixed_om, om_years, pv_fixed_om, annualized_capex_net_salv, total_annual_cost, econ_state


In [3]:
# =========================
# ADD-ON CELL: Economic Evaluation (NPV, B/C, Least Cost, Constant Annuity)
# This cell assumes you already have from prior cells:
#   - prj (Project)
#   - pv(), crf(), annuity_from_present()
#   - fixed_om, om_years, pv_fixed_om, annualized_capex_net_salv, total_annual_cost, econ_state
#
# It adds:
#   1) A benefit/avoided-cost stream (ENS*VOLL + congestion + avoided peaker)
#   2) NPV, Benefit/Cost Ratio, Least-Cost comparison, and Constant Annuity (EUAC)
#   3) A persistent dict econ_eval_state for future cells
# =========================

from dataclasses import dataclass

# ---------- Benefit model inputs (edit freely) ----------
# Reliability: avoided unserved energy (ENS) [MWh/year] and VOLL [$ per MWh]
ens_avoided_mwh_per_year = 250.0       # MWh/year
voll_per_mwh = 10_000.0               # $/MWh (i.e., $10/kWh)

# Congestion: avoided congestion payments or redispatch cost [$ per year]
avoided_congestion_per_year = 600_000.0

# Avoided diesel peaking: avoided running a diesel peaker for limited hours [$ per year]
avoided_diesel_peaking_per_year = 350_000.0

# Optional: benefit escalation (in real terms keep 0.0; nominal escalation could be >0)
benefit_escalation = 0.00  # per year (real)

# ---------- Helper functions ----------
def pv_cashflows(i: float, cashflows: dict) -> float:
    """
    Present value of cashflows keyed by integer year.
    Year 0 is undiscounted.
    """
    return sum(cf / ((1 + i) ** t) for t, cf in cashflows.items())

def build_uniform_series(payment_year1: float, i: float, T: int, escalation: float = 0.0) -> dict:
    """
    Build end-of-year cashflows from year 1..T.
    If escalation>0, cashflow grows geometrically in real/nominal terms (be consistent!).
    """
    return {t: payment_year1 * ((1 + escalation) ** (t - 1)) for t in range(1, T + 1)}

def euac_from_pv(PV_total: float, i: float, T: int) -> float:
    """Convert present value to equivalent uniform annual cost/benefit over T years."""
    return annuity_from_present(PV_total, i, T)

# ---------- Build BENEFIT stream ----------
reliability_benefit_per_year = ens_avoided_mwh_per_year * voll_per_mwh
benefit_per_year_year1 = (
    reliability_benefit_per_year
    + avoided_congestion_per_year
    + avoided_diesel_peaking_per_year
)

benefit_series = build_uniform_series(
    payment_year1=benefit_per_year_year1,
    i=prj.i,
    T=prj.T,
    escalation=benefit_escalation
)

PV_benefits = pv_cashflows(prj.i, benefit_series)

# ---------- Build COST stream for the same project ----------
# Costs included:
#   - CAPEX P0 at year 0
#   - reinvestment at reinv_year
#   - fixed O&M annually (years om_start..om_end)
#   - salvage as a negative cost (benefit) at year T

om_start_year, om_end_year = om_years
om_series = {t: fixed_om for t in range(om_start_year, om_end_year + 1)}

cost_cashflows = {0: prj.P0, prj.reinv_year: prj.reinv_cost}
cost_cashflows.update(om_series)
cost_cashflows[prj.T] = cost_cashflows.get(prj.T, 0.0) - prj.salvage  # salvage reduces net cost

PV_costs = pv_cashflows(prj.i, cost_cashflows)

# ---------- Economic evaluation metrics ----------
NPV = PV_benefits - PV_costs
BCR = (PV_benefits / PV_costs) if PV_costs != 0 else float("inf")

# Constant annuity (EUAC-style) for costs and benefits
EUAB = euac_from_pv(PV_benefits, prj.i, prj.T)     # Equivalent Uniform Annual BENEFIT
EUAC = euac_from_pv(PV_costs, prj.i, prj.T)        # Equivalent Uniform Annual COST
Net_Annual_Worth = EUAB - EUAC                      # "annualized NPV" intuition

# Least Cost (for now, single option): use EUAC as the "least-cost index"
least_cost_index = EUAC  # lower is better

# ---------- OPTIONAL: Add a second alternative to demonstrate Least Cost ----------
@dataclass
class Alternative:
    name: str
    P0: float
    reinv_year: int
    reinv_cost: float
    salvage: float
    fixed_om: float
    benefit_series: dict  # year-> $

def eval_alternative(alt: Alternative, i: float, T: int, om_years=(1, None)) -> dict:
    """Return PV costs, PV benefits, NPV, BCR, EUAC, EUAB for an alternative."""
    om_start, om_end = om_years
    if om_end is None:
        om_end = T

    # Costs
    cost_cf = {0: alt.P0, alt.reinv_year: alt.reinv_cost}
    cost_cf.update({t: alt.fixed_om for t in range(om_start, om_end + 1)})
    cost_cf[T] = cost_cf.get(T, 0.0) - alt.salvage
    PVc = pv_cashflows(i, cost_cf)

    # Benefits
    PVb = pv_cashflows(i, alt.benefit_series)

    return {
        "name": alt.name,
        "PV_costs": PVc,
        "PV_benefits": PVb,
        "NPV": PVb - PVc,
        "BCR": (PVb / PVc) if PVc != 0 else float("inf"),
        "EUAC": euac_from_pv(PVc, i, T),
        "EUAB": euac_from_pv(PVb, i, T),
        "Net_Annual_Worth": euac_from_pv(PVb, i, T) - euac_from_pv(PVc, i, T),
    }

# Define Alternative B (e.g., "smaller CAPEX but higher O&M"; benefits assumed identical for didactics)
altB = Alternative(
    name="Alt B (lower CAPEX, higher O&M)",
    P0=17_000_000.0,
    reinv_year=10,
    reinv_cost=0.35 * 17_000_000.0,   # 35% reinvestment
    salvage=0.08 * 17_000_000.0,      # 8% salvage
    fixed_om=650_000.0,               # higher O&M
    benefit_series=benefit_series     # same benefits for clean least-cost comparison on costs
)

altA = Alternative(
    name="Alt A (base case)",
    P0=prj.P0,
    reinv_year=prj.reinv_year,
    reinv_cost=prj.reinv_cost,
    salvage=prj.salvage,
    fixed_om=fixed_om,
    benefit_series=benefit_series
)

resA = eval_alternative(altA, prj.i, prj.T, om_years=om_years)
resB = eval_alternative(altB, prj.i, prj.T, om_years=om_years)

# Least-cost selection (cost-only) and best economic (NPV-based) selection
least_cost_choice = min([resA, resB], key=lambda r: r["EUAC"])["name"]
best_npv_choice = max([resA, resB], key=lambda r: r["NPV"])["name"]
best_bcr_choice = max([resA, resB], key=lambda r: r["BCR"])["name"]

# ---------- Persist state for future cells ----------
econ_eval_state = {
    "benefit_inputs": {
        "ens_avoided_mwh_per_year": ens_avoided_mwh_per_year,
        "voll_per_mwh": voll_per_mwh,
        "avoided_congestion_per_year": avoided_congestion_per_year,
        "avoided_diesel_peaking_per_year": avoided_diesel_peaking_per_year,
        "benefit_escalation": benefit_escalation,
        "benefit_per_year_year1": benefit_per_year_year1,
        "reliability_benefit_per_year": reliability_benefit_per_year,
    },
    "benefit_series": benefit_series,
    "cost_cashflows": cost_cashflows,
    "PV_benefits": PV_benefits,
    "PV_costs": PV_costs,
    "NPV": NPV,
    "BCR": BCR,
    "EUAB": EUAB,
    "EUAC": EUAC,
    "Net_Annual_Worth": Net_Annual_Worth,
    "least_cost_index_EUAC": least_cost_index,
    "alternatives": {"A": resA, "B": resB},
    "choices": {
        "least_cost_choice_by_EUAC": least_cost_choice,
        "best_choice_by_NPV": best_npv_choice,
        "best_choice_by_BCR": best_bcr_choice,
    }
}

print("=== Economic Evaluation Added (Benefits + Costs) ===")
print(f"Annual benefits (year 1): ${benefit_per_year_year1:,.0f}/yr")
print(f"  - Reliability (ENS*VOLL): ${reliability_benefit_per_year:,.0f}/yr "
      f"(ENS={ens_avoided_mwh_per_year:.1f} MWh/yr, VOLL=${voll_per_mwh:,.0f}/MWh)")
print(f"  - Avoided congestion:     ${avoided_congestion_per_year:,.0f}/yr")
print(f"  - Avoided diesel peaking: ${avoided_diesel_peaking_per_year:,.0f}/yr")
print()
print(f"PV(Benefits): ${PV_benefits:,.0f}")
print(f"PV(Costs):    ${PV_costs:,.0f}")
print(f"NPV:          ${NPV:,.0f}   (positive => economically attractive)")
print(f"B/C ratio:     {BCR:.3f}   (>1 => attractive)")
print()
print("Constant annuity (annual equivalents):")
print(f"EUAB (annual benefits): ${EUAB:,.0f}/yr")
print(f"EUAC (annual costs):    ${EUAC:,.0f}/yr")
print(f"Net Annual Worth:       ${Net_Annual_Worth:,.0f}/yr")
print()
print("Least-cost / selection demo with two alternatives:")
print(f" - {resA['name']}: EUAC=${resA['EUAC']:,.0f}/yr, NPV=${resA['NPV']:,.0f}, BCR={resA['BCR']:.3f}")
print(f" - {resB['name']}: EUAC=${resB['EUAC']:,.0f}/yr, NPV=${resB['NPV']:,.0f}, BCR={resB['BCR']:.3f}")
print()
print(f"Least-cost choice (min EUAC): {least_cost_choice}")
print(f"Best by NPV: {best_npv_choice}")
print(f"Best by BCR: {best_bcr_choice}")
print("\nSaved for future cells: benefit_series, cost_cashflows, PV_benefits, PV_costs, NPV, BCR, EUAB, EUAC, econ_eval_state")

=== Economic Evaluation Added (Benefits + Costs) ===
Annual benefits (year 1): $3,450,000/yr
  - Reliability (ENS*VOLL): $2,500,000/yr (ENS=250.0 MWh/yr, VOLL=$10,000/MWh)
  - Avoided congestion:     $600,000/yr
  - Avoided diesel peaking: $350,000/yr

PV(Benefits): $31,315,788
PV(Costs):    $23,900,076
NPV:          $7,415,712   (positive => economically attractive)
B/C ratio:     1.310   (>1 => attractive)

Constant annuity (annual equivalents):
EUAB (annual benefits): $3,450,000/yr
EUAC (annual costs):    $2,633,025/yr
Net Annual Worth:       $816,975/yr

Least-cost / selection demo with two alternatives:
 - Alt A (base case): EUAC=$2,633,025/yr, NPV=$7,415,712, BCR=1.310
 - Alt B (lower CAPEX, higher O&M): EUAC=$2,509,029/yr, NPV=$8,541,235, BCR=1.375

Least-cost choice (min EUAC): Alt B (lower CAPEX, higher O&M)
Best by NPV: Alt B (lower CAPEX, higher O&M)
Best by BCR: Alt B (lower CAPEX, higher O&M)

Saved for future cells: benefit_series, cost_cashflows, PV_benefits, PV_costs, N

In [4]:
# =========================
# ADD-ON CELL: LCOE from annualized cost (EUAC) / expected annual MWh
# Assumes existing variables from prior cells:
#   - prj
#   - econ_eval_state (contains EUAC)
#   - total_annual_cost (CAPEX annuity + fixed O&M) [optional]
# =========================

# --------- Generation assumptions (edit freely) ----------
plant_capacity_mw = 50.0          # MW nameplate
capacity_factor = 0.35            # average CF (0..1)
hours_per_year = 8760.0

# Optional: variable cost components (set to 0.0 if not yet teaching them)
variable_om_per_mwh = 0.0         # $/MWh
fuel_cost_per_mwh = 0.0           # $/MWh
carbon_cost_per_mwh = 0.0         # $/MWh (optional placeholder)

# --------- Expected annual energy ----------
expected_annual_mwh = plant_capacity_mw * capacity_factor * hours_per_year

# --------- Choose annualized cost basis ----------
# Preferred: EUAC computed from PV(costs) in econ_eval_state (includes CAPEX, reinvestment, fixed O&M, salvage)
EUAC_cost = econ_eval_state["EUAC"]

# Alternative (if you prefer to keep it strictly "CAPEX annuity + fixed O&M" from earlier cell):
EUAC_simple = total_annual_cost

# --------- LCOE calculations ----------
# Core LCOE using EUAC of costs (most consistent with your earlier PV-based economics cell)
lcoe_capex_fixed = EUAC_cost / expected_annual_mwh

# Add variable components if you want a "full" LCOE approximation
lcoe_total = lcoe_capex_fixed + variable_om_per_mwh + fuel_cost_per_mwh + carbon_cost_per_mwh

# Sensitivity helper: show how CF affects LCOE (optional quick scan)
def lcoe_from_cf(cf: float) -> float:
    mwh = plant_capacity_mw * cf * hours_per_year
    return EUAC_cost / mwh

# --------- Persist state for future cells ----------
lcoe_state = {
    "plant_capacity_mw": plant_capacity_mw,
    "capacity_factor": capacity_factor,
    "expected_annual_mwh": expected_annual_mwh,
    "EUAC_cost_used": EUAC_cost,
    "EUAC_simple": EUAC_simple,
    "lcoe_capex_fixed_$_per_mwh": lcoe_capex_fixed,
    "variable_om_per_mwh": variable_om_per_mwh,
    "fuel_cost_per_mwh": fuel_cost_per_mwh,
    "carbon_cost_per_mwh": carbon_cost_per_mwh,
    "lcoe_total_$_per_mwh": lcoe_total,
}

print("=== LCOE (Generation Expansion) ===")
print(f"Plant size: {plant_capacity_mw:.1f} MW, CF={capacity_factor:.2f}")
print(f"Expected annual energy: {expected_annual_mwh:,.0f} MWh/yr")
print(f"Annualized cost basis (EUAC from economics cell): ${EUAC_cost:,.0f}/yr")
print(f"LCOE (CAPEX+fixed O&M only, via EUAC/E): ${lcoe_capex_fixed:,.2f}/MWh")

if any(x != 0.0 for x in [variable_om_per_mwh, fuel_cost_per_mwh, carbon_cost_per_mwh]):
    print(f" + Variable O&M: ${variable_om_per_mwh:,.2f}/MWh")
    print(f" + Fuel:         ${fuel_cost_per_mwh:,.2f}/MWh")
    print(f" + Carbon:       ${carbon_cost_per_mwh:,.2f}/MWh")
    print(f"Total LCOE:      ${lcoe_total:,.2f}/MWh")

print("\nQuick CF sensitivity (same EUAC, different energy):")
for cf in [0.20, 0.30, 0.40, 0.50]:
    print(f"  CF={cf:.2f} -> LCOE=${lcoe_from_cf(cf):,.2f}/MWh")

print("\nSaved for future cells: expected_annual_mwh, lcoe_capex_fixed, lcoe_total, lcoe_state")

=== LCOE (Generation Expansion) ===
Plant size: 50.0 MW, CF=0.35
Expected annual energy: 153,300 MWh/yr
Annualized cost basis (EUAC from economics cell): $2,633,025/yr
LCOE (CAPEX+fixed O&M only, via EUAC/E): $17.18/MWh

Quick CF sensitivity (same EUAC, different energy):
  CF=0.20 -> LCOE=$30.06/MWh
  CF=0.30 -> LCOE=$20.04/MWh
  CF=0.40 -> LCOE=$15.03/MWh
  CF=0.50 -> LCOE=$12.02/MWh

Saved for future cells: expected_annual_mwh, lcoe_capex_fixed, lcoe_total, lcoe_state
